# <!-- TITLE --> __``` REAL-TIME OBJECT DETECTION ```__
#### <!-- AUTHOR --> - Author : __Dr. Amiehe-Essomba__ (c) 2023


In [ ]:
from demo.show_video import video 

video(video_path='./video/yolo_video2_pred.mp4')

## __```This notebook is only used for video prediction```__


> For video predictions please go [here](https://vision-api.streamlit.app/)

In [ ]:
from yolo.utils.tools import read_classes, read_anchors
import pandas as pd 
import seaborn as sns
from yolo.utils.tools import get_colors_for_classes
import random
import matplotlib.colors as mcolors
from random import sample
import numpy as np 
from demo.encapsulation import Wrapper 
from demo.demo import demo


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
np.random.seed(3)

#### __```Video Path```__

In [ ]:
path        = "./video/yolo_video2.mp4"

#### __```Reading all classes and anchors```__

In [ ]:
Class_names         = read_classes()
anchors             = read_anchors()

#### __```Hyper Parameters. You can change them```__

In [ ]:
classes         = []
max_boxes       = 10
class_names     = []
score_threshold = 0.2
iou_threshold   = 0.2
response        = False # show score
tracking        = False
tracker_type    = ("bytetrack.yaml", "botsort.yaml")
segmentation    = True

if segmentation is False:
    model_name  = "yolov8n.pt"

elif segmentation is True :
    model_name  = 'yolov8n-seg.pt'

elif segmentation is None:
    model_name = "license_plate_detector.pt"
else:
    model_name = "my_model"


#### __```Do not change this part of notebook.```__

In [ ]:
if class_names: pass 
else: class_names = Class_names.copy()

kwargs = {
    "class_names"       : Class_names, 
    "Class_names"       : Class_names,
    "max_boxes"         : max_boxes,
    "score_threshold"   : score_threshold,
    "iou_threshold"     : iou_threshold,
    "anchors"           : anchors
    }

#### __```Set color boxes. Do not change anything here```__

In [ ]:
class_names = kwargs['Class_names']
colors_     = get_colors_for_classes(len(class_names) + 10)

def f():
    s = random.sample(range(50), 1)
    return s[0] 

def g():
    num = random.sample(range(len(colors_)), len(class_names))
    return num 

colors      = {class_names[j] : colors_[i] if colors_[i] != (255, 255, 0) else colors_[j-1] for j, i in enumerate(g())}

In [ ]:
def scaling(image = None, shape = (608, 608), boxes = None, S = None):
    # Mettez à l'échelle l'image
    scaled_image    = image.resize(shape)
    new_width       = shape[0]
    new_height      = shape[1]
    # Mettez à l'échelle les boîtes englobantes
    scaling_factor_x = new_width / S[0]
    scaling_factor_y = new_height / S[1]

    scaled_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        scaled_x_min = int(x_min * scaling_factor_x)
        scaled_y_min = int(y_min * scaling_factor_y)
        scaled_x_max = int(x_max * scaling_factor_x)
        scaled_y_max = int(y_max * scaling_factor_y)
        scaled_boxes.append([scaled_x_min, scaled_y_min, scaled_x_max, scaled_y_max])

    return scaled_boxes, scaled_image

### __```Parameters```__

In [ ]:
model               = demo(model_name=model_name)
model.PATH          = path 
model.colors        = colors
model.model_name    = model_name
model.response      = response
model.youtube       = False 
model.save_file     = "video_prediction.mp4"
model.rescaling     = scaling

if tracking is True:
    model.tracker = tracker_type[0]

if segmentation is True:
    model.alpha = 30             # values in [0, 255]
    model.only_mask = True       
    model.mode  = "rgb"         # value in ['gray', 'rgb']
    model.with_names = True 
    

### __```About Video```__

In [ ]:
model.get_vido_info()

In [ ]:
model.duration_in_second = 10
model.video_step = 1
model.video_start = 0

### __```Prediction```__

In [ ]:
model.build_model(segmentation, **kwargs)

#### __```Using the dictionary generated by the prediction to see all classes include in the predicted image```__
----

In [ ]:
video(video_path=model.save_file)

In [ ]:
df          = model.df
data_frame  = pd.DataFrame(df)
data_frame.rename(columns={'label':'classes'}, inplace=True)
data_frame['count'] = [1 for i in range(len(data_frame.iloc[:, 0]))]


In [ ]:
data_frame = data_frame.groupby('classes').sum()
data_frame['labels'] = data_frame.index
data_frame.reset_index(drop=True, inplace=True)
display(data_frame)

#### __```Classes countplot```__

In [ ]:
# colors
rot             = 90.
python_colors   = np.array( list( mcolors.CSS4_COLORS.keys() ) )
length          = len(data_frame.labels)
indexes         = sample(range(python_colors.shape[0]), k=length)
unique_color    = list( python_colors[indexes] )
print(f"unique classes : {length}")


In [ ]:
plt.figure(figsize=(8,4))
plt.bar(height=list( data_frame['count'] ), x= range(length),  width=0.5, color=unique_color)
plt.xticks(range(length), list(data_frame.labels), rotation=rot, ha="center")
plt.show()

#### __```Data Frame details:```__

* classes
* scores
* bonding boxes 
* encoding classses

In [ ]:
display(data_frame.tail())

In [ ]:
import numpy as np
import cv2

# Création d'une image noire
height, width = 300, 400
image = np.zeros((height, width, 3), dtype=np.uint8)

# Définir les quatre points du quadrilatère
points = np.array([[50, 50], [150, 50], [200, 150], [100, 150]], np.int32)
points = points.reshape((-1, 1, 2))

# Tracer les lignes du quadrilatère (ne remplit pas encore)
cv2.polylines(image, [points], isClosed=True, color=(255, 255, 255), thickness=2)

# Remplir la région intérieure du quadrilatère
cv2.fillPoly(image, [points], color=(0, 255, 0))
plt.imshow(image)

<img width="180px" height="200px" src="https://i.pinimg.com/originals/18/e1/11/18e1110635dc82318910603571fe4e5a.jpg"></img>